In [22]:
import numpy as np
import json
layout_data = json.load(open("../../layout_new/layout_new.json", 'rb'))
frames = []

In [4]:
image_data = {"8":{"What is the animal?": [
            {
                "image_id": 524621,
                "question": "What is the animal?",
                "question_id": 524621007,
                "answer": "giraffe",
                "answer_type": "other",
                "target_img": {
                    "path": "val2014/",
                    "name": "COCO_val2014_000000524621.jpg"
                }
            },
            {
                "image_id": 27617,
                "question": "What is the animal?",
                "question_id": 27617005,
                "answer": "dog",
                "answer_type": "other",
                "target_img": {
                    "path": "val2014/",
                    "name": "COCO_val2014_000000027617.jpg"
                }
            },
            {
                "image_id": 166563,
                "question": "What is the animal?",
                "question_id": 166563002,
                "answer": "elephant",
                "answer_type": "other",
                "target_img": {
                    "path": "val2014/",
                    "name": "COCO_val2014_000000166563.jpg"
                }
            },
            {
                "image_id": 28872,
                "question": "What is the animal?",
                "question_id": 28872003,
                "answer": "horse",
                "answer_type": "other",
                "target_img": {
                    "path": "val2014/",
                    "name": "COCO_val2014_000000028872.jpg"
                }
            },
            {
                "image_id": 192730,
                "question": "What is the animal?",
                "question_id": 192730001,
                "answer": "cow",
                "answer_type": "other",
                "target_img": {
                    "path": "val2014/",
                    "name": "COCO_val2014_000000192730.jpg"
                }
            },
            {
                "image_id": 351297,
                "question": "What is the animal?",
                "question_id": 351297003,
                "answer": "giraffe",
                "answer_type": "other",
                "target_img": {
                    "path": "val2014/",
                    "name": "COCO_val2014_000000351297.jpg"
                }
            },
            {
                "image_id": 91564,
                "question": "What is the animal?",
                "question_id": 91564002,
                "answer": "sheep",
                "answer_type": "other",
                "target_img": {
                    "path": "val2014/",
                    "name": "COCO_val2014_000000091564.jpg"
                }
            },
            {
                "image_id": 509891,
                "question": "What is the animal?",
                "question_id": 509891002,
                "answer": "elephant",
                "answer_type": "other",
                "target_img": {
                    "path": "val2014/",
                    "name": "COCO_val2014_000000509891.jpg"
                }
            }
        ]}}

In [40]:

# Randomly selected ID(s)
unique_ids = [['8', 'What is the animal?']]

ids = [i[1] for i in unique_ids]
random_ids = ids

In [41]:
base_image_path = '../../VQA/'

In [ ]:
import matplotlib.pyplot as plt

for i in range(8):
    image_path = image_data[unique_ids[0][0]][unique_ids[0][1]][i]['target_img']
    image = plt.imread(base_image_path + image_path['path'] + image_path['name'])
    plt.imshow(image)
    plt.show()

In [43]:
import pandas as pd
df = pd.read_csv('../..//layouts/extracted_data.csv')


In [47]:
import math,cv2, os
import random
import numpy as np
# Load input images - assume we have a list of image paths in the correct order
test_set = []
set_source = 'VQA'
for q, im_set in enumerate(unique_ids):
    im_set = im_set[1]
    for num in range(1,9): # num of sub images
        extracted_rows = df[(df['Number of Subimages'] == num)]

        for qq in range(len(extracted_rows)):
            extracted_layouts = extracted_rows.iloc[qq]
            wall_gallery = np.ones((1000, 1000, 3), dtype=np.uint8) * 255

            test_data_ = {}
            test_data_['set_source'] = set_source
            layout = layout_data[str(num)][qq]

            for i in range(8):
                if not image_data['8'][im_set][i]['answer'] in [image_data['8'][im_set][j]['answer'] for j in range(8) if j != i]:
                    test_data_['qa_target_subimage_idx'] = i
                    break
            image_data['8'][im_set] = [image_data['8'][im_set][i]]+[image_data['8'][im_set][j] for j in range(8) if j != i]
            
            test_data_['layout'] = layout
            frames = layout['pos']
            images_list = image_data['8'][im_set]
            test_data_['sub_images'] = []
            image_paths = []
            
            path_to_idx = {}
            for p in range(len(images_list)):
                path_to_idx[images_list[p]['target_img']['path']] = []
            
            test_data_['sub_image_relations'] = {}

            for j in range(min(num, len(images_list))):
                sub_image_dict = {}
                # assert j==0 or 'target' in images_list[j]['source_img']['path']
                image_path = images_list[j]['target_img']['path']+'/'+images_list[j]['target_img']['name']
                image_paths.append(image_path)
                sub_image_dict['image_paths'] = image_path
                sub_image_dict['positional_descriptor'] = layout['positional_descriptor'][j]
                if sub_image_dict['positional_descriptor'].split('_')[-1] == str(extracted_layouts['ID for Subimage']):
                    test_data_['qa_target_subimage_idx'] = j
                if not images_list[j]['answer_type'] + '|' + images_list[j]['question'] + '|' + images_list[j]['answer'] in test_data_['sub_image_relations']:
                    test_data_['sub_image_relations'][images_list[j]['answer_type'] + '|' + images_list[j]['question'] + '|' + images_list[j]['answer']] = []
                test_data_['sub_image_relations'][images_list[j]['answer_type'] + '|' + images_list[j]['question'] + '|' + images_list[j]['answer']].append(j)
                
                test_data_['sub_images'].append(sub_image_dict)


            # Composite each image into the detected frames
            for (x1, y1, x2, y2), image_path in zip(frames, image_paths):
                # Load the image
                image = cv2.imread(os.path.join(base_image_path,image_path))
                
                # Calculate the aspect ratio of the image
                image_aspect_ratio = image.shape[1] / image.shape[0]
                
                # Calculate the aspect ratio of the frame
                frame_aspect_ratio = (x2 - x1) / (y2 - y1)
                
                # Determine the maximum size of the image that can fit in the frame while maintaining the aspect ratio
                if image_aspect_ratio > frame_aspect_ratio:
                    # The width of the image is the limiting factor
                    resized_width = x2 - x1
                    resized_height = math.floor(resized_width / image_aspect_ratio)
                else:
                    # The height of the image is the limiting factor
                    resized_height = y2 - y1
                    resized_width = math.floor(resized_height * image_aspect_ratio)
                
                # Resize the image to fit the frame
                resized_image = cv2.resize(image, (resized_width, resized_height))
                
                # Place the image in the frame
                x_offset = x1 + math.floor((x2 - x1 - resized_width) / 2)
                y_offset = y1 + math.floor((y2 - y1 - resized_height) / 2)
                wall_gallery[y_offset:y_offset+resized_height, x_offset:x_offset+resized_width] = resized_image

            # Save the composited image
            img_name = str(q)+ '_'+str(num)+'_'+extracted_layouts['Layout ID'].split('_')[1]+'.png'
            cv2.imwrite(img_name, wall_gallery)
            test_data_['composited_gallery'] = img_name
            
            test_set.append(test_data_)

In [45]:
image_path

'val2014//COCO_val2014_000000027617.jpg'